# Introduction
    * Understanding Data    
    * Outlier Analysis
    
    * Smote Sampling    
        * OverSampling with Smote
        * UnderSampling with Smote    
        
    * Diminsionality Reduction with Decesion Trees
    
    * Machine Learning
        * Decesion Trees
        * Neural Networks   
        * Checking Other 20 Classifiers at the same time
        * Ensembling Techniques
            * Bagging
            * Boasting
            * Stacking
            * Votting            



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

> # **UnderStanding Data**

In [ ]:
df=pd.read_csv("../input/train.csv")
df.drop("ID_code",axis=1,inplace=True)
print(df.head(5))
df.describe()

In [ ]:
import seaborn as sns
sns.heatmap(df.corr())

# **Missing Values**

In [ ]:
print("Total df Values",str(len(df)))
for i in df.columns:
    print(i,"\t\t",str(df[i].isnull().sum()))
    

In [ ]:
import matplotlib.pyplot as plt
target_0=len(df[df["target"]==0])
target_1=len(df[df["target"]==1])
# Data to plot
labels = df["target"].unique()
sizes = (target_0,target_1)

 
# Plot
plt.pie(sizes, labels=labels, 
autopct='%1.1f%%', shadow=True, startangle=140)
 
plt.axis('equal')
plt.show()

We have 90 percent 0 label values and 10 percent 1 label values 

# **Outliers Analysis**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
feature_names = df.columns
for i in range(len(feature_names)-1):
    figure = plt.figure()
    ax = sns.boxplot(x='target', y=feature_names[i], data=df)

**Detecting Outlier **

In [ ]:
# The diamond shaped dots outside the boxplot indicates the outliers 
# There are some extreme cases in K, Ba and Fe. 
# That why it is necessary to rule out the outliers 

df_old = df.copy(deep=True) # Make a copy of original data, just in case

# Create new dataframe for each type

types = df['target'].unique()
# seperating Dataframe by class Labels
d = {type: df[df['target'] == type] for type in types}

low = .25
high = .75

bounds = {}
for type in types:
    filt_df = d[type].loc[:, d[type].columns != 'target'] # Remove 'Type' Column
    quant_df = filt_df.quantile([low, high])
    IQR = quant_df.iloc[1,:]-  quant_df.iloc[0,:]
    quant_df.iloc[0,:] = quant_df.iloc[0,:] - 1.5*IQR
    quant_df.iloc[1,:] = quant_df.iloc[1,:] + 1.5*IQR
    bounds[type] = quant_df
    


In [ ]:
df_new = {}

for type in types:
    filt_df = d[type].loc[:, d[type].columns != 'target'] # Remove 'Type' Column
    filt_df = filt_df.apply(lambda x: x[(x>bounds[type].loc[low,x.name]) & (x < bounds[type].loc[high,x.name])], axis=0)
    df_new[type] = pd.concat([filt_df,d[type].loc[:,'target']], axis=1)

In [ ]:
df_OR=pd.concat(df_new)

In [ ]:
for i in range(len(feature_names)-1):
    figure = plt.figure()
    ax = sns.boxplot(x='target', y=feature_names[i], data=df_OR)

In [ ]:
len(df_old)


**Outliers Removed**

 # **Over Sampling**

[Smote oversampling](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/smote)

In [ ]:
X=df.drop("target",axis=1)
y=df["target"]
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


In [ ]:


# Data to plot
labels = df["target"].unique()
sizes = ([sum(y_train_res==0),sum(y_train_res==1)])

 
# Plot
plt.pie(sizes, labels=labels, 
autopct='%1.1f%%', shadow=True, startangle=140)
 
plt.axis('equal')
plt.show()

# Diminsionality reduction
WE have alot of diminsions here I am going to reduce some diminsions with the help of D-Trees

In [ ]:
#you can choose first 10 coloums to see or 100 or all coloums dynaimic code to chekc importance to
# fetures with decesion trees
coloums=[]
coloums_selected=200
for i in range (0,coloums_selected):
    coloums.append(i)
X_train_res=np.asarray(X_train_res)
X_test=np.asarray(X_test)
X_sliced_array_train=X_train_res[:20000,coloums]
X_sliced_array_test=X_test[:,coloums]


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
# i am taking 20000 exampling and just taking first 10 coloums
clf.fit(X_sliced_array_train,y_train_res[:20000])


In [ ]:
from sklearn.metrics import confusion_matrix
X_test=np.asarray(X_test)
y_pred=clf.predict(X_sliced_array_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

In [ ]:
from matplotlib.pyplot import figure

col=X_train.columns
feature_names = col[0:coloums_selected]
importance_frame = pd.DataFrame()
importance_frame['Features'] =  col[0:coloums_selected]
importance_frame['Importance'] =clf.feature_importances_
importance_frame = importance_frame.sort_values(by=['Importance'], ascending=True)
figure(num=None, figsize=(8, 40), dpi=80, facecolor='w', edgecolor='k')
plt.barh(coloums, importance_frame['Importance'], align='center', alpha=0.5)
plt.yticks(coloums, importance_frame['Features'])
plt.xlabel('Importance')
plt.title('Feature Importances')
plt.show()

In [ ]:
#now we have our Important Features
importance_frame

Selecting coloums having importance greater than 0.01

In [ ]:
df_selected_coloums=importance_frame[importance_frame['Importance']>0.01]
print("We have selected",str(len(df_selected_coloums)),"Coloums")
df_selected_coloums

# New Dataframe After Reduction

In [ ]:
Coloum_Names=["target"]
for i in df_selected_coloums.Features:
    Coloum_Names.append(i)
New_df=df.loc[ :, Coloum_Names]    
New_df.shape

**Lets see data Distribution**

In [ ]:
fig = plt.figure(figsize = (20, 25))
j = 0
#Droping_Characters and string coloums because graph donot support them

for i in New_df.columns:
    plt.subplot(7, 7, j+1)
    j += 1
    sns.distplot(New_df[i][New_df['target']==1], color='g', label = 'Positive')
    sns.distplot(New_df[i][New_df['target']==0], color='r', label = 'Negative')
    plt.legend(loc='best')
fig.suptitle('Standard Customer Transaction ')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

**Decesion Tree on Raw Data**

In [ ]:
X=New_df.drop("target",axis=1)
y=New_df["target"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
# i am taking 20000 exampling and just taking first 10 coloums
clf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

In [ ]:
coloums=[]
coloums_selected=len(X.columns)
for i in range(0,coloums_selected):
    coloums.append(i)
print(coloums_selected)
from matplotlib.pyplot import figure

col=New_df.columns
feature_names = col[1:coloums_selected+1]
importance_frame = pd.DataFrame()
importance_frame['Features'] =  col[1:coloums_selected+1]
importance_frame['Importance'] =clf.feature_importances_
importance_frame = importance_frame.sort_values(by=['Importance'], ascending=True)
figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
plt.barh(coloums, importance_frame['Importance'], align='center', alpha=0.5)
plt.yticks(coloums, importance_frame['Features'])
plt.xlabel('Importance')
plt.title('Feature Importances')
plt.show()

In [ ]:
len(clf.feature_importances_)

# **UnderSampling Data**

[Resource](https://www.kaggle.com/residentmario/undersampling-and-oversampling-imbalanced-data)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X=New_df.drop("target",axis=1)
y=New_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before UnderSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train==0)))


rus = RandomUnderSampler(random_state=0,ratio={0: 10000, 1:11000})
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_sample(X_train, y_train.ravel())

print('After UnderSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After UnderSampling, the shape of train_y: {} \n'.format( y_resampled.shape))

print("After UnderSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After UnderSampling, counts of label '0': {}".format(sum(y_resampled==0)))


colors = ['#05990d' if v == 0 else '#990606' if v == 1 else '#000000' for v in y_resampled]
plt.scatter(X_resampled[:, 0], X_resampled[:, 1], c=colors, linewidth=0.5, edgecolor='black')
sns.despine()
plt.title("RandomUnderSampler Output ($n_{class}=4700)$")
pass

#red is 0 
#green is 1

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

results=cross_val_score(clf,X_resampled ,y_resampled , cv=10)
print(results)
Average=sum(results) / len(results) 
print("Average Accuracy :",Average)

In [ ]:
from sklearn.metrics import confusion_matrix
clf.fit(X_resampled ,y_resampled)
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

# **OverSampling**

In [ ]:
X=New_df.drop("target",axis=1)
y=New_df["target"]
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

results=cross_val_score(clf,X_train_res, y_train_res  , cv=10)
print(results)
Average=sum(results) / len(results) 
print("Average Accuracy :",Average)

In [ ]:
from sklearn.metrics import confusion_matrix
clf.fit(X_train_res, y_train_res)
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

# **Raw Data **

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

results=cross_val_score(clf,X_train, y_train  , cv=10)
print(results)
Average=sum(results) / len(results) 
print("Average Accuracy :",Average)

In [ ]:
from sklearn.metrics import confusion_matrix
clf.fit(X_train, y_train)
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

**In our case Our Raw Data is performing well so we didnt go for Undersampling or Oversampling**

**Lets move forward and see our assumption of getting good scores on raw data is True or not**

# **Machine Learning**

In [ ]:
from sklearn import ensemble
from sklearn import gaussian_process
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import svm
from sklearn import tree
from sklearn import discriminant_analysis
from sklearn import model_selection
from xgboost.sklearn import XGBClassifier 
from sklearn import metrics

In [ ]:
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    #gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    #XGBClassifier()    
    ]




#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy(Scoring)',"Accuracy(Confusion Matrix)","true_positive","false_positive","true_negative","false_negative" ]
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
#ground Truth
MLA_compare.loc[row_index, 'MLA Name'] = "Ground Truth"
MLA_compare.loc[row_index, 'MLA Parameters'] = "NaN"
MLA_compare.loc[row_index, 'MLA Test Accuracy'] =0
MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =0
MLA_compare.loc[row_index, 'true_positive'] =sum(y_test==1)
MLA_compare.loc[row_index, 'true_negative'] =sum(y_test==0)
MLA_compare.loc[row_index, 'false_positive'] =0
MLA_compare.loc[row_index, 'false_negative'] =0
row_index+=1


#index through MLA and save performance to table

for alg in MLA:
    print("\n\nClassifier",alg)
    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
   # cv_results = model_selection.cross_validate(alg, X_train, y_train)
    alg.fit(X_train, y_train)
    y_pred=alg.predict(X_test)
    score=metrics.accuracy_score(y_test, y_pred)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] =score
    
    #COnfusion Matrix

    true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test, y_pred).ravel()
    Accuracy=(true_positive+true_negative)/(true_positive+false_positive+true_negative+false_negative)
    MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =Accuracy
    MLA_compare.loc[row_index, 'true_positive'] =true_positive
    MLA_compare.loc[row_index, 'true_negative'] =true_negative
    MLA_compare.loc[row_index, 'false_positive'] =false_positive
    MLA_compare.loc[row_index, 'false_negative'] =false_negative
    row_index+=1

    




In [ ]:
MLA_compare

# **Justification why Raw data is performing Well**
**Most of them are givig 89% accuracy **

**Our models are overfitting on negative Examples**

**My Assumption was wrong that Raw data is giving Better accuracy than UnderSampling and OverSampling because Our Models are classifying Negative examples very good. And we have alot of negative Examples in our Dataset Now we Have to do something Different**

**Its better to Experiment than make assumptions in machine Learnig**

# OverSampling data

I reapeat this step because at every step I am changing my X and Y. Some where I am putthing Raw data into X and Y and some wehre I am putting Undersampled data. So for confirming I repeat this step that read from data frame and perform the specefic step (over sample -- undersample ) on data

In [ ]:
X=New_df.drop("target",axis=1)
y=New_df["target"]
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [ ]:
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    #gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    #XGBClassifier()    
    ]




#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy(Scoring)',"Accuracy(Confusion Matrix)","true_positive","false_positive","true_negative","false_negative" ]
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
#ground Truth
MLA_compare.loc[row_index, 'MLA Name'] = "Ground Truth"
MLA_compare.loc[row_index, 'MLA Parameters'] = "NaN"
MLA_compare.loc[row_index, 'MLA Test Accuracy'] =0
MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =0
MLA_compare.loc[row_index, 'true_positive'] =sum(y_test==1)
MLA_compare.loc[row_index, 'true_negative'] =sum(y_test==0)
MLA_compare.loc[row_index, 'false_positive'] =0
MLA_compare.loc[row_index, 'false_negative'] =0
row_index+=1


#index through MLA and save performance to table

for alg in MLA:
    print("\n\nClassifier",alg)
    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
   # cv_results = model_selection.cross_validate(alg, X_train, y_train)
    alg.fit(X_train_res, y_train_res)
    y_pred=alg.predict(X_test)
    score=metrics.accuracy_score(y_test, y_pred)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] =score
    
    #COnfusion Matrix

    true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test, y_pred).ravel()
    Accuracy=(true_positive+true_negative)/(true_positive+false_positive+true_negative+false_negative)
    MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =Accuracy
    MLA_compare.loc[row_index, 'true_positive'] =true_positive
    MLA_compare.loc[row_index, 'true_negative'] =true_negative
    MLA_compare.loc[row_index, 'false_positive'] =false_positive
    MLA_compare.loc[row_index, 'false_negative'] =false_negative
    row_index+=1

    




# OverSampled Data Results

In [ ]:
MLA_compare

Now we have 50 percent positive examples and 50 percent negative examples 

# **Lets see now How Undersampling is working On our Data**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X=New_df.drop("target",axis=1)
y=New_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before UnderSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train==0)))


rus = RandomUnderSampler(random_state=0,ratio={0: 9000, 1:13000})
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_sample(X_train, y_train.ravel())

print('After UnderSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After UnderSampling, the shape of train_y: {} \n'.format( y_resampled.shape))

print("After UnderSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After UnderSampling, counts of label '0': {}".format(sum(y_resampled==0)))




#red is 0 
#green is 1

In [ ]:
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    #gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    #XGBClassifier()    
    ]




#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy(Scoring)',"Accuracy(Confusion Matrix)","true_positive","false_positive","true_negative","false_negative" ]
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
#ground Truth
MLA_compare.loc[row_index, 'MLA Name'] = "Ground Truth"
MLA_compare.loc[row_index, 'MLA Parameters'] = "NaN"
MLA_compare.loc[row_index, 'MLA Test Accuracy'] =0
MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =0
MLA_compare.loc[row_index, 'true_positive'] =sum(y_test==1)
MLA_compare.loc[row_index, 'true_negative'] =sum(y_test==0)
MLA_compare.loc[row_index, 'false_positive'] =0
MLA_compare.loc[row_index, 'false_negative'] =0
row_index+=1


#index through MLA and save performance to table

for alg in MLA:
    print("\n\nClassifier",alg)
    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
   # cv_results = model_selection.cross_validate(alg, X_train, y_train)
    alg.fit(X_resampled, y_resampled )
    y_pred=alg.predict(X_test)
    score=metrics.accuracy_score(y_test, y_pred)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] =score
    
    #COnfusion Matrix

    true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test, y_pred).ravel()
    Accuracy=(true_positive+true_negative)/(true_positive+false_positive+true_negative+false_negative)
    MLA_compare.loc[row_index, 'Accuracy(Confusion Matrix)'] =Accuracy
    MLA_compare.loc[row_index, 'true_positive'] =true_positive
    MLA_compare.loc[row_index, 'true_negative'] =true_negative
    MLA_compare.loc[row_index, 'false_positive'] =false_positive
    MLA_compare.loc[row_index, 'false_negative'] =false_negative
    row_index+=1

    




# Under Sampling Data Results

In [ ]:
MLA_compare

**From here I just Realized that I can achive better accuracy through neural netwrork(Perceptron, MLP) Classifier If I am able to do fine Tuning**

# Neural Networks

**Setting up Dataset**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from keras.utils.np_utils import to_categorical
X=New_df.drop("target",axis=1)
y=New_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before UnderSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train==0)))


rus = RandomUnderSampler(random_state=0,ratio={0: 9000, 1:13000})
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_sample(X_train, y_train)


X_resampled=np.asarray(X_resampled)
y_resampled=np.asarray(y_resampled)

y_resampled =to_categorical(y_resampled)
y_test=to_categorical(y_test)

y_resampled=y_resampled.astype("int64")
y_test=y_test.astype("int64")
print('After UnderSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After UnderSampling, the shape of train_y: {} \n'.format( y_resampled.shape))

print("After UnderSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After UnderSampling, counts of label '0': {}".format(sum(y_resampled==0)))




#red is 0 
#green is 1

# **Ramdom Search on MLP**

**I am just undersampling data to choose equal number of traning good sample rather than picking up random samples. It take less time to run my model for small example rather than whole dataset to set my hyper parameters this undersampling is only done for seeing the best parameters. Once we get good parameters we take that parameters and than check it on whole dataset.**

**For Just Taking Best Hyperpatameters**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X=New_df.drop("target",axis=1)
y=New_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before UnderSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before UnderSampling, counts of label '0': {} \n".format(sum(y_train==0)))


rus = RandomUnderSampler(random_state=0,ratio={0: 1000, 1:1100})
rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_sample(X_train, y_train.ravel())

print('After UnderSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After UnderSampling, the shape of train_y: {} \n'.format( y_resampled.shape))

print("After UnderSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After UnderSampling, counts of label '0': {}".format(sum(y_resampled==0)))


#red is 0 
#green is 1

**For Fitting our Dataset of Best Parameters**

In [ ]:
X=New_df.drop("target",axis=1)
y=New_df["target"]
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

#taking small sample for just selecting hyperparatmers


print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


In [ ]:
import math
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,BatchNormalization
from keras import losses
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras import regularizers

def best_parameter_model(lr=0.01,dropout=0.5,optimizer="adam",loss='mean_squared_error',
                    last_activation="softmax",activation="relu",clipnorm=0.1,
                    decay=1e-2,momentum=0.5,l1=0.01,l2=0.001,No_of_CONV_and_Maxpool_layers=3,
                    No_of_Dense_Layers=3,No_of_Units_in_dense_layers=24,Conv2d_filters=60):
    momentum=0.2  
    decay=0
    lr=0.0001 
    dropout=0.1
    optimizer='Adam'
    loss='mean_squared_error' 
    last_activation='softmax'
    activation='relu'
    clipnorm=1 
    l2=0.01 
    
    '''
    momentum=best["momentum"]
    decay=best["decay"]
    lr=best["lr"] 
    dropout=best["dropout"]
    optimizer=best["optimizer"]
    loss=best["loss"]
    last_activation=best["last_activation"]
    activation=best["activation"]
    clipnorm=best["clipnorm"]
    l2=best["l2"]
    '''
    #setting up loss fucntions
    loss=losses.mean_squared_error
    if(loss=='mean_squared_error'):
        loss=losses.mean_squared_error
    if(loss=="poisson"):
        loss=keras.losses.poisson
    if(loss=="mean_absolute_error"):
        loss=keras.losses.mean_absolute_percentage_error
    if(loss=="mean_squared_logarithmic_error"):
        loss=keras.losses.mean_squared_logarithmic_error
    if(loss=="binary_crossentropy"):
        loss=keras.losses.binary_crossentropy
    if(loss=="hinge"):
        loss=keras.losses.hinge
        
    #setting up Optimizers
    opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    if optimizer=="RMSprop":
        opt=keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
    if optimizer=="Adamax":
        opt=keras.optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
        
        
    #last activation layer
    if last_activation=="softmax":
        last_activation=keras.activations.softmax
    if last_activation=="sigmoid":
        last_activation=keras.activations.sigmoid
    
    
    
    
    
    model=Sequential()
    
    
    
    

    model.add(Dense(units=84,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=62,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=31,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(units=2,activation=last_activation))

    model.compile(loss=loss,optimizer=opt,
                     metrics=['accuracy'])
    
    model.compile(loss=loss ,optimizer=opt,
                 metrics=['accuracy'])
    
    

    return model

In [ ]:
params = {'lr': (0.0001, 0.01,0.0009,0.001,0.002 ),
     'epochs': [500,100,1000,1500],
     'dropout': (0, 0.2,0.4, 0.8),
     'optimizer': ['Adam','Adagrad','sgd','RMSprop','Adamax'],
     'loss': ["binary_crossentropy","mean_squared_error","hinge","mean_absolute_error","mean_squared_logarithmic_error","poisson"],
     'last_activation': ["softmax","sigmoid"],
     'activation' :["relu","selu","linear","sigmoid"],
     'clipnorm':(0.0,0.5,1),
     'decay':(1e-6,1e-4,1e-8),
     'momentum':(0.9,0.5,0.2),
     'l1': (0.01,0.001,0.0001),
     'l2': (0.01,0.001,0.0001),
     'No_of_Dense_Layers': [4,6,8,10,12,14]
     }


In [ ]:

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
# model class to use in the scikit random search CV 

model = KerasClassifier(build_fn=best_parameter_model, epochs=10, batch_size=500, verbose=1)
RandomizedSearchfit = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=params, 
                          verbose=1,  n_iter=10, n_jobs=1)


In [ ]:
#having some problem in this line
from keras.utils import to_categorical
y_resampled=to_categorical(y_resampled)
RandomizedSearch_result = RandomizedSearchfit.fit(X_resampled, y_resampled )


** Having Some problem in random search of MLP if you guys get the solution please let me inform Thank you!**

**Kindly See the hide cells to take my Problem Under Considration**

[Link To Stack OverFlow](https://stackoverflow.com/questions/55145261/randomized-search-valued-passed-value-passed-to-parameter-shape-has-datatype-f)



# **Random Search on Random Forest**

In [ ]:


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier


# build a classifier
clf = RandomForestClassifier(n_estimators=20)



# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
             "n_estimators":[100,500,1000,2000,5000],
             }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5,verbose=1)

#Already Searched Parameters
#random_search.fit(X_resampled, y_resampled )
#clf = RandomForestClassifier(max_depth=random_search.best_params_['max_depth'],
 #                            max_features=random_search.best_params_['max_depth'],min_samples_split=random_search.best_params_['min_samples_split'],
  #                          bootstrap=random_search.best_params_['bootstrap'],
   #                          criterion=random_search.best_params_['criterion'],
    #                         n_estimators=random_search.best_params_['n_estimators']
     #                        ,random_state=0)
#print(random_search.best_params_)

best params are
Get through Random Search

clf = RandomForestClassifier(n_estimators= 1000, min_samples_split= 2, max_features= 3,
                             max_depth= None, criterion= 'gini', bootstrap= True
                             ,random_state=0)

**Now I a going to fit my dataset on best parameters thats why Increasing the size of dataset by Oversampling**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(n_estimators= 1000, min_samples_split= 2, max_features= 3,
                             max_depth= None, criterion= 'gini', bootstrap= True
                             ,random_state=0)
clf.fit(X_train_res, y_train_res)
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

**Accuracy is increased but we can Improve more accuracy lets try some other classifier**

# **Random Search On Extra Tree Classifier**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree


clfETC=tree.ExtraTreeClassifier()


param_dist = {"max_depth": [3,5,8,13,17,None],
              "max_features": [0.01,0.03,0.07,0.10,0.15,0.20,"auto"],
              "min_samples_split": [2, 5, 10,15,20],
              
              "criterion": ["gini", "entropy"],
              "splitter" :['best','random'],
             
             }

n_iter_search = 20
random_search = RandomizedSearchCV(clfETC, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=10,verbose=1)

#Already Searched Parameter
#random_search.fit(X_resampled, y_resampled)
#print(random_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clfET = tree.ExtraTreeClassifier( min_samples_split= 2, max_features= 0.01,max_depth= 8,
                              criterion= 'entropy')
                            

from sklearn.metrics import confusion_matrix
clfET.fit(X_train_res, y_train_res)
X_test=np.asarray(X_test)
y_pred=clf.predict(X_test)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test,y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Accuracy=(true_positive+true_negative)/(true_positive+false_negative+true_negative+false_positive)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)

**Extra trees are not performing well after hyperparameter Optimization**

# **Ensembling Methods**

# **Bagging**

A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

In [ ]:
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier

bstlfy=ensemble.BaggingClassifier(base_estimator=clf,n_estimators=5)
bstlfy=bstlfy.fit(X_train_res, y_train_res)
prediction=bstlfy.predict(X_test)
    

true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test, prediction).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")

Accuracy=(true_positive+true_negative)/(true_positive+false_positive+true_negative+false_negative)
print("Accuracy: ",Accuracy)

Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

        #FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)